In [12]:
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
files = []
for file in os.listdir("../data/embeddings"):
    files.append(os.path.join("../data/embeddings", file))
files.sort(key=os.path.getmtime)

In [3]:
audio_embeddings = []
for file in files:
    with open(file, "rb") as fp:
        audio_embeddings.extend(pickle.load(fp))
        
audio_embeddings = np.array([embedding.detach().numpy() for embedding in audio_embeddings])
audio_embeddings.shape

(7442, 1, 768)

In [4]:
audio_embeddings = audio_embeddings.reshape((7442, 768))

In [5]:
emotion_categories = ['NEU', 'HAP', 'SAD', 'ANG', 'FEA', 'DIS']

data_dir = "../data/CREMA-D/"
audio_labels = []

for file in os.listdir(data_dir):
    for emotion in emotion_categories:
        if emotion in file:
            audio_labels.append(emotion)
            
label_encoder = LabelEncoder()
label_encoder.fit(emotion_categories)
labels = label_encoder.transform(audio_labels)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(audio_embeddings, labels, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.33, random_state=42)

In [7]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [10]:
y_pred = clf.predict(X_val)

In [14]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.63      0.53       237
           1       0.41      0.35      0.38       272
           2       0.34      0.32      0.33       253
           3       0.34      0.34      0.34       239
           4       0.48      0.38      0.43       244
           5       0.35      0.36      0.36       251

    accuracy                           0.40      1496
   macro avg       0.40      0.40      0.39      1496
weighted avg       0.40      0.40      0.39      1496



In [22]:
y_pred = clf.predict(X_train)

In [23]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       912
           1       1.00      1.00      1.00       885
           2       1.00      1.00      1.00       900
           3       1.00      1.00      1.00       893
           4       1.00      1.00      1.00       734
           5       1.00      1.00      1.00       885

    accuracy                           1.00      5209
   macro avg       1.00      1.00      1.00      5209
weighted avg       1.00      1.00      1.00      5209

